In [1]:
%%capture
!pip install koreanize-matplotlib
import koreanize_matplotlib

In [2]:
import pandas as pd
exchange = pd.read_csv('/content/exchange_final.csv')
m2 = pd.read_csv('/content/M2_2000_2024.CSV' , encoding='cp949')

In [3]:
# 1. 날짜 포맷 맞추기
exchange['TIME'] = pd.to_datetime(exchange['TIME']).dt.strftime('%Y-%m')

# 2. m2 인코딩해서 불러오기
m2 = pd.read_csv('/content/M2_2000_2024.CSV', encoding='cp949')  # 또는 euc-kr

# 3. 조인 (왼쪽 기준: exchange 기준으로 붙이기)
merged = pd.merge(exchange, m2, on='TIME', how='left')

In [4]:
import random
import numpy as np
import tensorflow as tf
import os

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)

In [5]:
# 유의미한 변수
features = ['경제 심리 지수', '한국 외환 보유액', 'vix', 'WTI 유가', '한국 실업률', '두바이 유가',
                        'Brent 유가', '천연가스 가격', '유연탄 가격', '미국 주가지수',
                        '한국 인플레이션율', '인플레이션 격차', '실질GDP성장률','M2',
                        '금 가격', '구리 가격', '아연 가격', '알루미늄 가격', '니켈 가격',
                        '외국인 투자 금액']

In [6]:
target = ['원/달러환율']

In [20]:
# 모델링/통계
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import pandas as pd
import numpy as np

In [29]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 77 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     300 non-null    int64  
 1   TIME           300 non-null    object 
 2   원/달러환율         300 non-null    float64
 3   한국 장기 시장 금리    291 non-null    float64
 4   미국 장기 시장 금리    300 non-null    float64
 5   중국 장기 시장 금리    132 non-null    float64
 6   한국 단기 시장 금리    300 non-null    float64
 7   미국 단기 시장 금리    299 non-null    float64
 8   중국 단기 시장 금리    299 non-null    float64
 9   전체 수출 총액       300 non-null    int64  
 10  미국 수출 총액       300 non-null    int64  
 11  중국 수출 총액       300 non-null    int64  
 12  전체 수입 총액       300 non-null    int64  
 13  미국 수입 총액       300 non-null    int64  
 14  중국 수입 총액       300 non-null    int64  
 15  소득 교역 조건 지수    300 non-null    float64
 16  해외 직접 투자 금액    300 non-null    int64  
 17  경제 심리 지수       264 non-null    float64
 18  한국 기준 금리  

## 피처 선택 결과 다름

In [23]:
def feature_selection_and_evaluation(method_name, selector, is_mlxtend=False):
    import numpy as np

    all_features = ['경제 심리 지수', '한국 외환 보유액', 'vix', 'WTI 유가', '한국 실업률',
                    '두바이 유가', 'Brent 유가', '천연가스 가격', '유연탄 가격', '미국 주가지수',
                    '한국 인플레이션율', '인플레이션 격차', '실질GDP성장률','M2',
                    '금 가격', '구리 가격', '아연 가격', '알루미늄 가격', '니켈 가격',
                    '외국인 투자 금액']

    df = merged.dropna(subset=all_features + ['원/달러환율'])
    X = df[all_features]
    y = df['원/달러환율']

    if is_mlxtend:
        selector.fit(X.values, y.values)
        selected_features = [all_features[i] for i in selector.k_feature_idx_]
    else:
        selector.fit(X, y)
        selected_features = list(X.columns[selector.get_support()])

    print(f"\n[{method_name}] 선택된 features: {selected_features}")
    return selected_features

In [24]:
# 전진 선택법
selector_forward = SequentialFeatureSelector(LinearRegression(), n_features_to_select='auto', direction='forward')
features_forward = feature_selection_and_evaluation("전진 선택법", selector_forward, is_mlxtend=False)

# 후진 선택법
selector_backward = SequentialFeatureSelector(LinearRegression(), n_features_to_select='auto', direction='backward')
features_backward = feature_selection_and_evaluation("후진 선택법", selector_backward, is_mlxtend=False)

# 단계적 선택법 (Stepwise - mlxtend)
selector_stepwise = SFS(
    LinearRegression(),
    k_features='best',
    forward=True,
    floating=True,
    scoring='r2',
    cv=0
)
features_stepwise = feature_selection_and_evaluation("단계적 선택법", selector_stepwise, is_mlxtend=True)


[전진 선택법] 선택된 features: ['두바이 유가', 'Brent 유가', '유연탄 가격', '한국 인플레이션율', '인플레이션 격차', '실질GDP성장률', 'M2', '금 가격', '아연 가격', '외국인 투자 금액']

[후진 선택법] 선택된 features: ['한국 외환 보유액', 'WTI 유가', '두바이 유가', 'Brent 유가', '천연가스 가격', '유연탄 가격', '미국 주가지수', 'M2', '아연 가격', '니켈 가격']

[단계적 선택법] 선택된 features: ['경제 심리 지수', '한국 외환 보유액', 'vix', 'WTI 유가', '한국 실업률', '두바이 유가', 'Brent 유가', '천연가스 가격', '유연탄 가격', '미국 주가지수', '한국 인플레이션율', '인플레이션 격차', '실질GDP성장률', 'M2', '금 가격', '구리 가격', '아연 가격', '알루미늄 가격', '니켈 가격', '외국인 투자 금액']


# 피처

### 유의미한 피처 중 선택법 적용

In [27]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

# 선택된 피처 리스트
feature_sets = {
    "전진 선택법": [
        'vix', '한국 실업률', '두바이 유가', 'Brent 유가', '천연가스 가격',
        '미국 주가지수', '인플레이션 격차', 'M2', '금 가격', '니켈 가격'
    ],
    "후진 선택법": [
        '한국 실업률', '두바이 유가', 'Brent 유가', '유연탄 가격',
        '미국 주가지수', '인플레이션 격차', '실질GDP성장률', 'M2', '금 가격', '구리 가격'
    ],
    "단계적 선택법": [
        '경제 심리 지수', '한국 외환 보유액', 'vix', 'WTI 유가', '한국 실업률',
        '두바이 유가', 'Brent 유가', '천연가스 가격', '유연탄 가격', '미국 주가지수',
        '한국 인플레이션율', '인플레이션 격차', '실질GDP성장률', 'M2', '금 가격',
        '구리 가격', '아연 가격', '알루미늄 가격', '니켈 가격', '외국인 투자 금액'
    ]
}

# 시퀀스 생성 함수
def create_sequences(X, y, look_back=12):
    X_seq, y_seq = [], []
    for i in range(len(X) - look_back):
        X_seq.append(X[i:i+look_back])
        y_seq.append(y[i+look_back])
    return np.array(X_seq), np.array(y_seq)

# 성능 저장용 리스트
results = []

# 반복 실행
for method, features in feature_sets.items():
    df = merged.dropna(subset=features + ['원/달러환율'])
    X = df[features].values
    y = df[['원/달러환율']].values

    X_seq, y_seq = create_sequences(X, y, look_back=12)

    train_size = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:train_size], X_seq[train_size:]
    y_train, y_test = y_seq[:train_size], y_seq[train_size:]

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_train_2d = X_train.reshape(-1, X_train.shape[2])
    X_test_2d = X_test.reshape(-1, X_test.shape[2])

    X_train_scaled = scaler_x.fit_transform(X_train_2d).reshape(X_train.shape)
    X_test_scaled = scaler_x.transform(X_test_2d).reshape(X_test.shape)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_test_scaled = scaler_y.transform(y_test)

    model = Sequential([
        GRU(16, input_shape=(X_train.shape[1], X_train.shape[2])),
        Dropout(0.2),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_scaled, y_train_scaled,
              validation_data=(X_test_scaled, y_test_scaled),
              epochs=100, batch_size=16,
              callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
              verbose=0)

    y_train_pred = scaler_y.inverse_transform(model.predict(X_train_scaled))
    y_test_pred = scaler_y.inverse_transform(model.predict(X_test_scaled))
    y_train_true = scaler_y.inverse_transform(y_train_scaled)
    y_test_true = scaler_y.inverse_transform(y_test_scaled)

    results.append({
        '선택법': method,
        'Train RMSE': np.sqrt(mean_squared_error(y_train_true, y_train_pred)),
        'Test RMSE': np.sqrt(mean_squared_error(y_test_true, y_test_pred)),
        'Train MAE': mean_absolute_error(y_train_true, y_train_pred),
        'Test MAE': mean_absolute_error(y_test_true, y_test_pred),
        'Train R2': r2_score(y_train_true, y_train_pred),
        'Test R2': r2_score(y_test_true, y_test_pred)
    })

# 결과 출력
results_df = pd.DataFrame(results)
print(results_df)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
       선택법  Train RMSE   Test RMSE   Train MAE    Test MAE  Train R2   Test R2
0   전진 선택법  127.654537  157.882358  108.499300  130.405882 -0.640654 -1.780285
1   후진 선택법   96.763299  163.762610   77.289017  126.010635  0.057317 -1.991243
2  단계적 선택법   44.050401   65.294018   33.832628   50.215450  0.802175  0.464002


## 모든 피처에서 선택법 적용

In [31]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np
import pandas as pd

# 이미지 기반 feature set 구성
feature_sets = {
    "전진 선택법": [
        '경제 심리 지수', '한국 실업률', '미국 주가지수', '천연가스 가격',
        '알루미늄 가격', '니켈 가격', '한국 인플레이션율', '실질GDP성장률',
        'vix', '외국인 투자 금액'
    ],
    "후진 선택법": [
        '경제 심리 지수', '두바이 유가', 'Brent 유가', '유연탄 가격',
        '구리 가격', '알루미늄 가격', '니켈 가격', '금 가격',
        '실질GDP성장률', 'vix'
    ],
    "단계적 선택법": [
        '경제 심리 지수', '한국 외환 보유액', 'vix', 'WTI 유가', '한국 실업률',
        '두바이 유가', 'Brent 유가', '천연가스 가격', '유연탄 가격', '미국 주가지수',
        '한국 인플레이션율', '인플레이션 격차', '실질GDP성장률', 'M2', '금 가격',
        '구리 가격', '아연 가격', '알루미늄 가격', '니켈 가격', '외국인 투자 금액', '한국 상품수지'
    ]
}

def create_sequences(X, y, look_back=12):
    X_seq, y_seq = [], []
    for i in range(len(X) - look_back):
        X_seq.append(X[i:i+look_back])
        y_seq.append(y[i+look_back])
    return np.array(X_seq), np.array(y_seq)

results = []

for method, features in feature_sets.items():
    df = merged.dropna(subset=features + ['원/달러환율'])
    X = df[features].values
    y = df[['원/달러환율']].values

    X_seq, y_seq = create_sequences(X, y, look_back=12)

    train_size = int(len(X_seq) * 0.8)
    X_train, X_test = X_seq[:train_size], X_seq[train_size:]
    y_train, y_test = y_seq[:train_size], y_seq[train_size:]

    scaler_x = MinMaxScaler()
    scaler_y = MinMaxScaler()

    X_train_2d = X_train.reshape(-1, X_train.shape[2])
    X_test_2d = X_test.reshape(-1, X_test.shape[2])

    X_train_scaled = scaler_x.fit_transform(X_train_2d).reshape(X_train.shape)
    X_test_scaled = scaler_x.transform(X_test_2d).reshape(X_test.shape)

    y_train_scaled = scaler_y.fit_transform(y_train)
    y_test_scaled = scaler_y.transform(y_test)

    model = Sequential([
    GRU(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.1),
    Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    model.fit(X_train_scaled, y_train_scaled,
              validation_data=(X_test_scaled, y_test_scaled),
              epochs=100, batch_size=16,
              callbacks=[EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)],
              verbose=0)

    y_train_pred = scaler_y.inverse_transform(model.predict(X_train_scaled))
    y_test_pred = scaler_y.inverse_transform(model.predict(X_test_scaled))
    y_train_true = scaler_y.inverse_transform(y_train_scaled)
    y_test_true = scaler_y.inverse_transform(y_test_scaled)

    results.append({
        '선택법': method,
        'Train RMSE': np.sqrt(mean_squared_error(y_train_true, y_train_pred)),
        'Test RMSE': np.sqrt(mean_squared_error(y_test_true, y_test_pred)),
        'Train MAE': mean_absolute_error(y_train_true, y_train_pred),
        'Test MAE': mean_absolute_error(y_test_true, y_test_pred),
        'Train R2': r2_score(y_train_true, y_train_pred),
        'Test R2': r2_score(y_test_true, y_test_pred)
    })

results_df = pd.DataFrame(results)
print(results_df)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
       선택법  Train RMSE   Test RMSE  Train MAE    Test MAE  Train R2   Test R2
0   전진 선택법  104.482172   89.127601  73.932078   76.358637 -0.160775  0.097415
1   후진 선택법   81.789952   92.527863  68.291285   77.774777  0.286969  0.090223
2  단계적 선택법  109.315741  164.849717  85.277865  138.148776 -0.218277 -2.416591
